# Test delete

In [ ]:
from lndb_setup import delete, load, login, set_storage
from lndb_setup._settings_load import load_instance_settings
from lndb_setup._settings_store import instance_settings_file
from lndb_setup._hub import (
    connect_hub_with_auth,
    get_instance,
    get_storage_by_id,
    get_user_by_handle,
    get_instance_default_storage,
)
from pathlib import Path

In [ ]:
hub = connect_hub_with_auth()
testuser1 = get_user_by_handle(hub, "testuser1")

## Delete an instance

### As a non instance owner

In [ ]:
login("testuser2")

You can only delete an instance if you are the owner of this one:

In [ ]:
delete("mydata") == "instance-settings-not-found"

### As the instance owner

In [ ]:
login("testuser1")

In [ ]:
settings_file = instance_settings_file("mydata", "testuser1")
instance_default_storage = get_instance_default_storage(hub, "mydata", testuser1["id"])

Running 
```
lndb delete mydata --owner testuser1
```

on the command line runs the following:

In [ ]:
delete("mydata")

In [ ]:
assert Path(instance_default_storage["root"]).exists() == False
assert get_instance(hub, "mydata", testuser1["id"]) is None
assert get_storage_by_id(hub, instance_default_storage["id"]) is None
assert settings_file.exists() == False

### For instance with remote default storage

In [ ]:
instance = get_instance(hub, "lndb-setup-ci", testuser1["id"])
instance_default_storage = get_instance_default_storage(
    hub, "lndb-setup-ci", testuser1["id"]
)

In [ ]:
delete("lndb-setup-ci")

In [ ]:
assert get_storage_by_id(hub, instance_default_storage["id"]) is None
assert Path(instance["cache_dir"]).exists() == False

### For instance with local shared default storage

In [ ]:
load("mydata_instance_name")
set_storage("mydatapg")

In [ ]:
instance_default_storage = get_instance_default_storage(hub, "pgtest", testuser1["id"])

In [ ]:
delete("pgtest")

In [ ]:
assert get_storage_by_id(hub, instance_default_storage["id"]) is not None

In [ ]:
hub.auth.sign_out()